<a href="https://colab.research.google.com/github/lolddong/data_analysis/blob/main/21_IMDB_RNN(not_run).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB RNN

## 0. 텍스트 분석 알고리즘
### 1) 텍스트 분석을 위해 개선된 알고리즘의 종류
#### RNN
- Embedding 층은 단순하게 데이터의 표현을 학습하여 데이터 사전을 구축하는 것
- 유사한 의미의 단어를 비슷한 공간에 매핑할 수 있지만, 시퀀스 데이터의 중요한 특성인 순서와 맥락까지 고려한 것은 아니다
- 순환 신경망은 이 문제를 해결하기 위해 고안된 층
- 완전연결층, 컨볼루션 신경망과 반대되는 개념
- 완전연결층과 컨볼루션 신경망은 피드 포워드 네트워크(feed-forward network)라고 표현
- `피드 포워드 네트워크`는 신경망이 가지는 `모든 출력값`이 마지막층인 `출력층을 향한다`
- 하지만 `순환 신경망`은 각 층의 결과값이 출력층을 향하면서도 동시에 `현재 층의 다음 계산에 사용`된다

#### LSTM
- RNN의 그래디언트 손실문제를 보완한 방법
- 정보를 여러 시점에 걸쳐 나르는 장치('Cell State')가 추가되었다
- 이로 인해 그래디언트를 보존할 수 있어 그래디언트 손실 문제가 발생하지 않도록 도와준다

#### GRN
- 게이트 메커니즘이 적용된 RNN의 일종으로 LSTM에서 영감을 받았으며 더 간략한 구조를 갖는다
- 한국인 조경현 박사님이 제안한 방법

### 2) 텍스트 분석 알고리즘 적용하기
- 전체 소스코드는 지금까지의 예제들과 동일하게 진행된다
> 1. 패키지 준비 → 2. 데이터셋 준비 → 3. 데이터 전처리 → 4. 탐색적 데이터 분석(문자열 토큰화, 데이터를 동일한 길이로 맞추기) → 5. 데이터 셋 분할 → 6. 모델 개발(정의+학습) → 7. 학습 결과 평가 → 8. 학습결과 적용

- 이 과정에서 학습 모델을 정의하는 부분에서 적용할 알고리즘만 변경하면 되기에 여기서는 RNN을 먼저 적용해 본 후, 학습 모델을 LSTM과 GRU로 각각변경하여 다시 학습을 수행해 보도록 한다
- 학습 시간이 매우 오래 걸리는 예제이므로 가급적 GPU가 탑재된 컴퓨터에서 실습하는 것이 좋다

## 1. 패키지 준비하기

In [1]:
import helper
from matplotlib import pyplot as plt
from pandas import DataFrame
import seaborn as sb
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb

from sklearn.model_selection import train_test_split
form sklearn.metrics import confusion_matrix

## 2. 데이터셋 준비

In [2]:
# 가장 빈번하게 사용되는 단어의 갯수
num_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

print(f'훈련 데이터 {x_train.shape} 레이블 {y_train.shape}')
print(f'검증 데이터 {x_test.shape} 레이블 {y_test.shape}')

17464789/17464789 [==============================] - 0s 0us/step
훈련 데이터 (25000,) 레이블 (25000,)
검증 데이터 (25000,) 레이블 (25000,)


## 3. 데이터 전처리
### 1) 데이터를 동일한 길이로 맞추기 (padding)

In [3]:
# 최대 문장 길이
max_len = 500
print('Before pad_sequences: ', len(x_train[0]), len(x_train[1]))
pad_x_train = pad_sequences(x_train, maxlen=max_len, padding='pre')
pad_x_test =  pad_sequences(x_test, maxlen=max_len, padding='pre')

# 원래 단어의 앞에 '지정해준 단어의 길이 - 원해 단어의 길이' (500-218)만큼 0이 추가된 것을 볼 수 있다
print('After pad_sequences: ', len(pad_x_train[0]), len(pad_x_train[1]))

print(pad_x_train[0])

Before pad_sequences:  218 189
After pad_sequences:  500 500
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 

## 4. 데이터셋 분할하기
- 이미 데이터셋이 훈련데이터와 검증데이터로 분리되어 있기 때문에 별도의 분할 작업을 수행할 필요는 없다.

## 5. 모델 개발
### 1) 모델 정의 (RNN)
  - 파라미터:
    - `return_sequences`:
      - True ->  모든 학습 시점의 은닉상태를 출력해줌
      - False -> 마지막 시점의 은닉 상태만 출력 - default는 False
    - dropout:
      - 지정된 비율만큼 학습을 건너뛰게 하는 파라미터
      - 이 파라미터를 사용하면 과거 학습 정보를 잃어버릴 확률이 높아지고 그에 따라 모델 성능이 나빠질 가능성이 있다
    - recurrent_dropout(순환드롭아웃):
      - 과거 학습정보를 잃어버리는 문제를 해결하기 위해 적용하는 옵션

In [4]:
my_model = Sequential()
my_model.add(Embedding(input_dim = num_words,
                       output_dim = 32,
                       input_length = max_len))
my_model.add(SimpleRNN(32,
                       return_sequences=True,
                       dropout=0.15,
                       recurrent_dropout=0.15))
my_model.add(SimpleRNN(16))
my_model.add(Dense(1, activation='sigmoid'))

# 결국은 긍정, 부정을 분류하는 문제이므로 이진분류에 해당한다
my_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['acc'])
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 500, 32)           2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 16)                784       
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 322881 (1.23 MB)
Trainable params: 322881 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 2) 학습하기

In [5]:
result = my_model.fit(pad_x_train,
                      y_train,
                      epochs=500,
                      validation_data=(pad_x_test, y_test),
                      callbacks=[EarlyStopping(monitor='val_loss',
                                               patience=5,
                                               verbose=1),
                                 ReduceLROnPlateau(monitor='val_loss',
                                                   patience=3,
                                                   factor=0.5,
                                                   min_lr=0.0001,
                                                   verbose=1)])

Epoch 1/500
782/782 [==============================] - 1117s 1s/step - loss: 0.6837 - acc: 0.5543 - val_loss: 0.6669 - val_acc: 0.6235 - lr: 0.0010
Epoch 2/500
  9/782 [..............................] - ETA: 15:48 - loss: 0.6788 - acc: 0.5486

KeyboardInterrupt: ignored

## 6. 학습결과 평가

In [ ]:
# 강사님 helper 코드
# helper.tf_result_plot(result)

# evaluate1 = my_model.evaluate(pad_x_train, y_train)
# print('최종 훈련 손실률: %f, 최종 훈련 정확도: %f' % (evaluate1[0], evaluate1[1]))
# evaluate2 = my_model.evaluare(pad_x_test, y_test)
# print('최종 검증 손슬률: %f, 최종 검증 정확도: %f' % (evaluate2[0], evaluate2[1]))

In [ ]:
# 학습 결과
result_df = DataFrame(result.history)
result_df['epochs'] = result_df.index+1
result_df.set_index('epochs', inplace = True)
result_df

In [ ]:
# Colab 그래프 한글글꼴 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm-/.cache/matplotlib -rf

from matplotlib import font_manager as fm
import matplotlib.pyplot as plt
fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
                  name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})

In [ ]:
# 그래프 기본설정
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# 그래프를 그리기 위한 객체 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), dpi=150)

# 1) 훈련 및 검증 손실 그리기
sb.lineplot(x=result_df.index,
            y='loss',
            data=result_df,
            color='lightblue',
            label='훈련 손실률',
            ax=ax1)
sb.lineplot(x=result_df.index,
            y='val_loss',
            data=result_df,
            color='lightcoral',
            label='검증 손실률',
            ax = ax1)
ax1.set_title('훈련 및 검증 손실률')
ax1.set_xlabel('반복회차')
ax1.set_ylabel('손실률')
ax1.grid()
ax1.legend()

# 2) 훈련 및 검증 정확도 그리기
sb.lineplot(x=result_df.index,
            y='acc',
            data=result_df,
            color='lightblue',
            label='훈련 정확도',
            ax=ax2)
sb.lineplot(x=result_df.index,
            y='val_acc',
            data=result_df,
            color='lightcoral',
            label='검증 정확도',
            ax=ax2)
ax2.set_title('훈련 및 검증 정확도')
ax2.set_xlabel('반복회차')
ax2.set_ylabel('정확도')
ax2.grid()
ax2.legend()

plt.show()
plt.close()

In [ ]:
evaluate1 = my_model.evaluate(pad_x_train, y_train)
print('최종 훈련 손실률: %f, 최종 훈련 정확도: %f' % (evaluate1[0], evaluate1[1]))
evaluate2 = my_model.evaluate(pad_x_test, y_test)
print('최종 검증 손실률: %f, 최종 검증 정확도: %f' % (evaluate2[0], evaluate2[1]))

## 7. 학습 결과 적용
### 1) 훈련 데이터에 대한 예측 결과 산정

In [ ]:
result = my_model.predict(pad_x_train)
data_count, case_count = result.shape
print('%d개의 훈련 데이터가 %d개의 경우의 수를 갖는다' % (data_count, case_count))
result

### 2) 예측 결과를 1차원 배열로 변환

In [ ]:
f_results = result.flatten()
f_results

### 3) 훈련데이터의 실제 결과값과 머신러닝에 의한 예측값 비교

In [ ]:
kdf = DataFrame({
    'train': y_train,
    'pred': np.round(f_results)
})

kdf['pred'] = kdf['pred'].astype('int')
cm = confusion_matrix(kdf['train'], kdf['pred'])
plt.figure(figsize=(7, 3))
sb.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('RNN Result')
plt.show()
plt.close()